# Serving Models

This notebook contains a PyTorch model trained on the Iris dataset.  We will be using this model throughout the remainder of the model serving material.

In [ ]:
# Install required python packages
%%capture
! pip install mlflow

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn
import mlflow
import mlflow.pyfunc
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Mount our gdrive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Download the csv to the content directory in colab
# You can see the csv by opening the file explorer tab on the left of the screen
# You may need to click the refresh button at the top of the file explorer window
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2020-03-13 20:56:40--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2020-03-13 20:56:40 (86.8 MB/s) - ‘iris.csv’ saved [3975/3975]



# Data Prep

In [ ]:
# Load the csv into a pandas dataframe and inspect the data
iris_df = pd.read_csv('/content/iris.csv')
iris_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [ ]:
# Initialize a label encoder for the class names
# This is an example of a data artifact that will be required at inference time
# Any data preprocessing artifacts should be packaged with the corresponding model
label_encoder = preprocessing.LabelEncoder()

# Convert labels to ints
iris_df['variety'] = label_encoder.fit_transform(iris_df['variety'])

In [ ]:
# Label breakdown - we no longer have strings for class names
iris_df['variety'].value_counts()

2    50
1    50
0    50
Name: variety, dtype: int64

In [ ]:
# Inspect the data after label encoding
print(iris_df.shape)
iris_df.head()

(150, 5)


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [ ]:
# Drop labels from out training features
iris_x = iris_df.drop('variety', axis = 1)
iris_y = iris_df[['variety']]

# Generate train / test splits for training and evaluation
X_train, x_test, Y_train, y_test = train_test_split(iris_x,
                                                    iris_y,
                                                    test_size=0.3,
                                                    random_state=0)

In [ ]:
# Convert to tensors
X_train = torch.from_numpy(X_train.values).float()
X_test = torch.from_numpy(x_test.values).float()
y_train = torch.from_numpy(Y_train.values).view(1,-1)[0]
y_test = torch.from_numpy(y_test.values).view(1,-1)[0]

# Model

Models are typically created within a notebook but the model class should be separated out into its own Python file for packaging purposes.  Notebooks are great for experimenting with ideas but it can be a challenge to then take that code and structure it properly in a project format.



In [ ]:
# PLEASE WATCH THE ACCOMPANYING VIDEO FOR THIS NOTEBOOK
# NOTE: REMEBER IN THE VIDEO WE SEPERATED OUT OUR MODEL CLASS
# INTO ITS OWN MODEL.PY FILE

# We will eventually pull this model out into its own Python file for packaging

import torch
import torch.nn as nn
from torch.nn import functional as F

input_size = 4
output_size = 3
hidden_size = 30

class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)


# Since we move our model class to model.py
# We can import it from the local file system
# This files is included in the material

# from model import IrisNet

In [ ]:
# Initialize the network
model = IrisNet()

# Set the optamizer and loss function
optimizer = optim.Adam(model.parameters(), lr = 0.03)
loss_fn = nn.NLLLoss()

In [ ]:
# Train the model
epochs = 500

for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_fn(y_pred , y_train)
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch: {epoch} loss: {loss.item()}')

Epoch: 0 loss: 1.100169062614441
Epoch: 100 loss: 0.024682074785232544
Epoch: 200 loss: 0.017290666699409485
Epoch: 300 loss: 0.013995449990034103
Epoch: 400 loss: 0.01133726630359888


In [ ]:
def inference(model, user_input):
    """
    Conduct inference for a model
    Args:
      model (torch): An instance of a torch model
      user_input (tensor): User provided input strucutred as a tensor
    Returns:
      Predicted labels [(str)]
    """

    # Get prediction
    pred = torch.argmax(model(user_input), dim=1)

    # Given the predicted integer, find the label from the label encoder
    pred_labels = label_encoder.inverse_transform(pred)

    return pred_labels

In [ ]:
# Take a Setosa sample from X_test
example = torch.tensor([[5.1, 3.5, 1.4, 0.2]])

# Inference
pred = inference(model, example)
print(pred)

['Setosa']


# MLflow PyTorch Example

In [ ]:
import mlflow.pytorch

mlflow_pytorch_path = '/content/gdrive/My Drive/MLOPS/hands_on/models/iris_mlflow_pytorch'

In [ ]:
# Default Conda ENV
mlflow.pytorch.get_default_conda_env()

{'channels': ['defaults', 'pytorch'],
 'dependencies': ['python=3.6.9',
  'pytorch=1.4.0',
  'torchvision=0.5.0',
  {'pip': ['mlflow', 'cloudpickle==1.2.2']}],
 'name': 'mlflow-env'}

In [ ]:
# Let's create our own conda environment
conda_env = {
    'channels': ['defaults', 'pytorch'],
    'dependencies': [
      f'python=3.6.9',
      {
          'pip':[
            f'mlflow=={mlflow.__version__}',
            f'scikit-learn=={sklearn.__version__}',
            'torch==1.4.0',
            'cloudpickle==1.2.2'
          ]
      }
    ],
    'name': 'mlflow-env-iris'
}

In [ ]:
# Save the model
mlflow.pytorch.save_model(model, mlflow_pytorch_path, conda_env=conda_env)

In [ ]:
# Load model
new_model = mlflow.pytorch.load_model(mlflow_pytorch_path)

# Take a Setosa sample from X_test
example = torch.tensor([[5.1, 3.5, 1.4, 0.2]])

# Get prediction
pred = torch.argmax(new_model(example), dim=1)

print(pred)

tensor([0])


# Serializing data artifacts

We will need to serialize a few data artifacts for this model:



1.   Our PyTorch models state_dict
2.   The label encoder used for transforming ints to their corresponding strings



In [ ]:
import pickle

In [ ]:
# Serialize the label encoder
# This will be required at inference time
le_path = '/content/label_encoder.pkl'
with open(le_path, 'wb') as handle:
    pickle.dump(label_encoder, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Serialize the models state_dict
state_dict_path = f'/content/state_dict.pt'
torch.save(model.state_dict(), state_dict_path)

In [ ]:
# Inspect the data artifacts
! ls /content

gdrive	iris.csv  label_encoder.pkl  mlflow_pytorch  sample_data  state_dict.pt


# MLflow PyFunc Packaging

Now that we have everything serialized to disk it's time to package everything up together.

In [ ]:
# Here we will create an artifacts object
# It will contain all of the data artifacts that we want to package with the model
artifacts = {
    "state_dict": state_dict_path,
    "label_encoder": le_path
}

# This will serve as an MLflow wrapper for the model
class ModelWrapper(mlflow.pyfunc.PythonModel):

    # Load in the model and all required artifacts
    # The context object is provided by the MLflow framework
    # It will contain all of the artifacts specified above
    def load_context(self, context):
        import torch
        import pickle
        from model import IrisNet

        # Initialize the model and load in the state dict
        self.model = IrisNet()
        self.model.load_state_dict(torch.load(context.artifacts["state_dict"]))

        # Load in and deserialize the label encoder object
        with open(context.artifacts["label_encoder"], 'rb') as handle:
            self.label_encoder = pickle.load(handle)

    # Create a predict function for our models
    def predict(self, context, model_input):
      
        example = torch.tensor(model_input.values)
        pred = torch.argmax(model(example.float()), dim=1)
        pred_labels = self.label_encoder.inverse_transform(pred)
        return pred_labels

In [ ]:
# Inspect the default conda environment for MLflow
mlflow.pyfunc.get_default_conda_env()

{'channels': ['defaults'],
 'dependencies': ['python=3.6.9', {'pip': ['mlflow', 'cloudpickle==1.2.2']}],
 'name': 'mlflow-env'}

In [ ]:
# PLEASE NOTE: I HAVE CHANGED THE DICT STRUCTURE SLIGHTY FROM THE VIDEO

# Let's create our own conda environment
conda_env = {
    'channels': ['defaults', 'pytorch'],
    'dependencies': [
      f'python=3.6.9',
      {
          'pip':[
            f'mlflow=={mlflow.__version__}',
            f'scikit-learn=={sklearn.__version__}',
            'torch==1.4.0',
            'cloudpickle==1.2.2'
          ]
      }
    ],
    'name': 'mlflow-env-iris'
}

In [ ]:
# Location in our gdrive where we want the model to be saved
mlflow_pyfunc_model_path = f"/content/gdrive/My Drive/MLOPS/hands_on/models/iris_model_pyfunc"

# Package the model!
mlflow.pyfunc.save_model(path=mlflow_pyfunc_model_path,
                         python_model=ModelWrapper(),
                         artifacts=artifacts,
                         conda_env=conda_env,
                         code_path=['/content/model.py', '/content/meta_data.txt'])

# Test Importing

In [ ]:
import pandas as pd

# Load the model in `python_function` format
loaded_model = mlflow.pyfunc.load_model(mlflow_pyfunc_model_path)

# Evaluate the model
test_predictions = loaded_model.predict(pd.DataFrame([[5.1, 3.5, 1.4, 0.2]]))

print(test_predictions)

['Setosa']
